In [34]:
import numpy as np
from scipy.optimize import minimize
from scipy.interpolate import interp1d

In [35]:
def topsis(decision_matrix, weights, criteria):
    """
    Implementation of the TOPSIS method for multicriteria decision analysis in discrete form.

    Parameters:
    decision_matrix (ndarray): Matrix (m x n) with m alternatives and n criteria.
    weights (list or ndarray): Weights for each criterion.
    criteria (list or ndarray): A list specifying whether each criterion should be maximized (1) or minimized (-1).

    Returns:
    tuple: Ranking of alternatives and their respective scores.
    """
    # Step 1: Normalize the decision matrix
    norm_matrix = decision_matrix / np.sqrt((decision_matrix ** 2).sum(axis=0))

    # Step 2: Apply weights to the normalized matrix
    weighted_matrix = norm_matrix * weights

    # Step 3: Determine the ideal (best) and anti-ideal (worst) solutions
    ideal_best = np.max(weighted_matrix, axis=0) * (criteria == 1) + np.min(weighted_matrix, axis=0) * (criteria == -1)
    ideal_worst = np.min(weighted_matrix, axis=0) * (criteria == 1) + np.max(weighted_matrix, axis=0) * (criteria == -1)

    # Step 4: Calculate distances from the ideal and anti-ideal solutions
    dist_to_ideal = np.sqrt(((weighted_matrix - ideal_best) ** 2).sum(axis=1))
    dist_to_worst = np.sqrt(((weighted_matrix - ideal_worst) ** 2).sum(axis=1))

    # Step 5: Calculate the score for each alternative
    score = dist_to_worst / (dist_to_ideal + dist_to_worst)

    # Step 6: Rank the alternatives
    ranking = np.argsort(score)[::-1]  # Sort in descending order by score
    return ranking, score

In [36]:
def fuzzy_topsis(decision_matrix, weights, criteria_type):
    """
    Implementation of the Fuzzy TOPSIS method for multicriteria decision analysis.

    Parameters:
    decision_matrix (ndarray): Matrix (m x n x 3) with m alternatives, n criteria, and 3 values representing fuzzy numbers (low, medium, high).
    weights (list or ndarray): Weights for each criterion.
    criteria_type (list): A list specifying whether each criterion should be 'max' or 'min'.

    Returns:
    ndarray: Score for each alternative, ranking them from best to worst.
    """
    # Step 1: Normalize the decision matrix
    norm_matrix = np.zeros_like(decision_matrix, dtype=float)
    for j in range(decision_matrix.shape[1]):
        if criteria_type[j] == 'max':
            max_value = np.max(decision_matrix[:, j, 2])
            norm_matrix[:, j, 0] = decision_matrix[:, j, 0] / max_value
            norm_matrix[:, j, 1] = decision_matrix[:, j, 1] / max_value
            norm_matrix[:, j, 2] = decision_matrix[:, j, 2] / max_value
        else:
            min_value = np.min(decision_matrix[:, j, 0])
            norm_matrix[:, j, 0] = min_value / decision_matrix[:, j, 2]
            norm_matrix[:, j, 1] = min_value / decision_matrix[:, j, 1]
            norm_matrix[:, j, 2] = min_value / decision_matrix[:, j, 0]

    # Step 2: Apply the weights
    weighted_matrix = np.zeros_like(norm_matrix, dtype=float)
    for j in range(norm_matrix.shape[1]):
        weighted_matrix[:, j, 0] = norm_matrix[:, j, 0] * weights[j]
        weighted_matrix[:, j, 1] = norm_matrix[:, j, 1] * weights[j]
        weighted_matrix[:, j, 2] = norm_matrix[:, j, 2] * weights[j]

    # Step 3: Determine FPIS and FNIS
    fpis = np.zeros((decision_matrix.shape[1], 3), dtype=float)
    fnis = np.zeros((decision_matrix.shape[1], 3), dtype=float)
    for j in range(weighted_matrix.shape[1]):
        if criteria_type[j] == 'max':
            fpis[j] = np.max(weighted_matrix[:, j, :], axis=0)
            fnis[j] = np.min(weighted_matrix[:, j, :], axis=0)
        else:
            fpis[j] = np.min(weighted_matrix[:, j, :], axis=0)
            fnis[j] = np.max(weighted_matrix[:, j, :], axis=0)

    # Step 4: Calculate the distances to FPIS and FNIS
    distance_to_fpis = np.sqrt(np.sum(np.power(weighted_matrix - fpis, 2), axis=(1, 2)))
    distance_to_fnis = np.sqrt(np.sum(np.power(weighted_matrix - fnis, 2), axis=(1, 2)))

    # Step 5: Calculate the similarity to the ideal solution
    with np.errstate(divide='ignore', invalid='ignore'):
        similarity_to_ideal = distance_to_fnis / (distance_to_fpis + distance_to_fnis)
        similarity_to_ideal[np.isnan(similarity_to_ideal)] = 0  # Handle division by zero

    return similarity_to_ideal

In [37]:
def continuous_reference_set_method(objective_functions, directions, a, b, bounds, x0=None):
    """
    Implementacja metody zbiorów odniesienia dla ciągłego problemu optymalizacji wielokryterialnej.

    Parametry:
    - objective_functions: lista funkcji celu
    - directions: numpy array z wartościami 1 (maksymalizacja) lub -1 (minimalizacja) dla każdego kryterium
    - a: numpy array z punktami aspiracji dla każdego kryterium
    - b: numpy array z punktami status quo dla każdego kryterium
    - bounds: lista krotek określających ograniczenia dla zmiennych decyzyjnych
    - x0: opcjonalnie, punkt startowy dla optymalizacji

    Zwraca:
    - result: wynik optymalizacji z biblioteki scipy.optimize
    - optimal_f_values: wartości funkcji celu w optymalnym punkcie
    - optimal_score: wartość funkcji skoringowej w optymalnym punkcie
    """

    # Liczba kryteriów
    num_criteria = len(objective_functions)

    # Funkcja agregująca wartości funkcji celu
    def F(x):
        return np.array([f(x) for f in objective_functions])

    # Funkcja normalizująca z uwzględnieniem kierunku optymalizacji
    def normalize(f_values, a, b, directions):
        norm_values = np.zeros_like(f_values)
        for i in range(len(f_values)):
            if directions[i] == -1:  # Minimalizacja
                norm_values[i] = (f_values[i] - a[i]) / (b[i] - a[i])
            else:  # Maksymalizacja
                norm_values[i] = (b[i] - f_values[i]) / (b[i] - a[i])
        return norm_values

    # Funkcja skoringowa oparta na metodzie TOPSIS
    def scoring_function(x):
        f_values = F(x)
        # Normalizacja z uwzględnieniem kierunku optymalizacji
        f_norm = normalize(f_values, a, b, directions)
        # Obliczanie odległości od punktu idealnego (aspiracji) i anty-idealnego (status quo)
        distance_to_a = np.linalg.norm(f_norm - np.zeros(len(f_norm)))
        distance_to_b = np.linalg.norm(f_norm - np.ones(len(f_norm)))
        # Wskaźnik bliskości do ideału
        C = distance_to_b / (distance_to_a + distance_to_b)
        return C

    # Funkcja do minimalizacji (negacja wskaźnika C)
    def objective(x):
        return -scoring_function(x)

    # Punkt startowy
    if x0 is None:
        x0 = np.zeros(len(bounds))

    # Optymalizacja
    result = minimize(
        objective,
        x0=x0,
        bounds=bounds,
        method='SLSQP'
    )

    optimal_x = result.x
    optimal_f_values = F(optimal_x)
    optimal_score = scoring_function(optimal_x)

    return result, optimal_f_values, optimal_score

In [38]:
def discrete_reference_set_method(alternatives, directions, a, b):
    """
    Implementacja metody zbiorów odniesienia dla dyskretnego problemu optymalizacji wielokryterialnej.

    Parametry:
    - alternatives: numpy array z listą alternatyw (rozwiązań), gdzie każdy wiersz to alternatywa, a kolumny to kryteria.
    - directions: numpy array z wartościami 1 (maksymalizacja) lub -1 (minimalizacja) dla każdego kryterium.
    - a: numpy array z punktami aspiracji (najlepsze wartości) dla każdego kryterium.
    - b: numpy array z punktami status quo (najgorsze wartości) dla każdego kryterium.

    Zwraca:
    - ranking: numpy array z indeksami alternatyw posortowanych od najbardziej preferowanych do najmniej.
    - scores: numpy array z wartościami funkcji skoringowej dla każdej alternatywy.
    """

    # Funkcja normalizująca z uwzględnieniem kierunku optymalizacji
    def normalize(alternatives, a, b, directions):
        norm_alt = np.zeros_like(alternatives, dtype=float)
        for i in range(len(directions)):
            if directions[i] == -1:  # Minimalizacja
                norm_alt[:, i] = (alternatives[:, i] - a[i]) / (b[i] - a[i])
            else:  # Maksymalizacja
                norm_alt[:, i] = (b[i] - alternatives[:, i]) / (b[i] - a[i])
        return norm_alt

    # Funkcja skoringowa
    def scoring_function(alternatives, a, b, directions):
        norm_alt = normalize(alternatives, a, b, directions)
        distance_to_a = np.linalg.norm(norm_alt - np.zeros((len(alternatives), len(a))), axis=1)
        distance_to_b = np.linalg.norm(norm_alt - np.ones((len(alternatives), len(a))), axis=1)
        C = distance_to_b / (distance_to_a + distance_to_b)
        return C

    # Obliczanie wartości funkcji skoringowej
    scores = scoring_function(alternatives, a, b, directions)

    # Tworzenie rankingu
    ranking = np.argsort(-scores)  # Negacja, aby sortować malejąco

    return ranking, scores

In [39]:
# Funkcja do wyznaczania funkcji użyteczności dla wersji dyskretnej
def UTA_function_discrete(values, reference_points, lambda_param=1):
    """
    Aproksymuje funkcje użyteczności dla wersji dyskretnej.

    :param values: Lista wartości alternatyw w przestrzeni kryteriów
    :param reference_points: Lista punktów referencyjnych (najlepszy, najgorszy, neutralny)
    :param lambda_param: Współczynnik λ (preferencje decydenta)
    :return: Lista funkcji użyteczności dla wszystkich alternatyw
    """
    utility_values = []

    # Iterujemy przez alternatywy
    for val in values:
        alternative_utility = []

        # Iterujemy przez każde kryterium
        for i in range(len(val)):
            ref_values = [ref[i] for ref in reference_points]  # Zbiór punktów referencyjnych dla kryterium i
            # Interpolacja liniowa dla każdego kryterium
            interp_func = interp1d(ref_values, [0, 1, 0], kind='linear', fill_value="extrapolate")
            utility_value = interp_func(val[i]) * lambda_param  # Użycie λ
            alternative_utility.append(utility_value)

        utility_values.append(alternative_utility)

    return np.array(utility_values)

In [40]:
# Funkcja do porównania alternatyw (dyskretna wersja)
def UTAstar_discrete(values, reference_points, lambda_param=1):
    """
    Metoda UTA Star dla wersji dyskretnej (wielokryterialnej) uwzględniająca preferencje decydenta.

    :param values: Lista wartości alternatyw w przestrzeni kryteriów
    :param reference_points: Lista punktów referencyjnych dla każdego kryterium
    :param lambda_param: Współczynnik λ (preferencje decydenta)
    :return: Wybrana alternatywa
    """
    # Wyznaczanie funkcji użyteczności
    utility_values = UTA_function_discrete(values, reference_points, lambda_param)

    # Wybór rozwiązania kompromisowego - wybieramy alternatywę z maksymalną sumą użyteczności
    summed_utility = np.sum(utility_values, axis=1)
    best_solution_idx = np.argmax(summed_utility)  # Wybieramy alternatywę z największą sumą funkcji użyteczności
    return best_solution_idx, utility_values

In [41]:
# Funkcja do wyznaczania funkcji użyteczności (ciągła wersja z preferencjami)
def UTA_function_continuous(values, reference_points, lambda_param=1, use_polynomial=False):
    """
    Aproksymuje funkcje użyteczności w przestrzeni ciągłej dla wielu kryteriów.
    Uwzględnia preferencje decydenta przez współczynnik λ i punkty referencyjne.

    :param values: Lista wartości kryteriów dla alternatyw
    :param reference_points: Lista punktów referencyjnych (np. [najlepszy, najgorszy, neutralny])
    :param lambda_param: Współczynnik λ (preferencje decydenta)
    :param use_polynomial: Jeśli True, stosuje aproksymację wielomianową, w przeciwnym razie interpolację
    :return: Funkcje użyteczności
    """
    utility_values = []
    for i in range(len(reference_points[0])):  # Iterujemy po kryteriach
        ref_values = [ref[i] for ref in reference_points]  # Zbiór punktów referencyjnych dla kryterium i
        if use_polynomial:
            # Przy użyciu funkcji aproksymacji wielomianowej
            coef = np.polyfit(ref_values, [0, 1, 0], 2)  # Przykład: funkcja kwadratowa
            poly_func = np.poly1d(coef)
            utility_values.append(poly_func(values[:, i]) * lambda_param)  # Użycie λ
        else:
            # Interpolacja liniowa
            interp_func = interp1d(ref_values, [0, 1, 0], kind='linear', fill_value="extrapolate")
            utility_values.append(interp_func(values[:, i]) * lambda_param)  # Użycie λ
    return np.array(utility_values).T  # Zwracamy funkcje użyteczności dla wszystkich alternatyw

In [42]:
# Funkcja do porównania alternatyw (ciągła wersja z uwzględnieniem preferencji)
def UTAstar_continuous(values, reference_points, lambda_param=1, use_polynomial=False):
    """
    Metoda UTA Star dla wersji ciągłej (wielokryterialnej) uwzględniająca preferencje decydenta.

    :param values: Lista wartości alternatyw w przestrzeni kryteriów
    :param reference_points: Lista punktów referencyjnych dla każdego kryterium
    :param lambda_param: Współczynnik λ (preferencje decydenta)
    :param use_polynomial: Jeśli True, stosuje aproksymację wielomianową, w przeciwnym razie interpolację
    :return: Wybrana alternatywa
    """
    # Wyznaczanie funkcji użyteczności
    utility_values = UTA_function_continuous(values, reference_points, lambda_param, use_polynomial)

    # Wybór rozwiązania kompromisowego - wybieramy alternatywę z maksymalną sumą użyteczności
    summed_utility = np.sum(utility_values, axis=1)
    best_solution = np.argmax(summed_utility)  # Wybieramy alternatywę z największą sumą funkcji użyteczności
    return best_solution, utility_values